In [1]:
from spectral import envi
import numpy as np
from tkinter import Tk,filedialog
from argparse import ArgumentParser
import os
import time
import shutil
import pandas as pd
import ntpath
from timeit import default_timer as timer

In [2]:
#Function to obtain the namefile for the csv and hdf
def path_leaf(tgtname):
    head, tail = ntpath.split(tgtname)
    return tail or ntpath.basename(head)

The hyperspectral cube is read from the hdr file relative to a CRISM image. In this case from a calibreted I/F MTRDR.
To read the hdr file, i used the module envi from the spectral library.
The hyperspectral cube is opened as a masked numpy 3-D array [width, height, wavelength]. then 

In [3]:
def img2np(hdrfile):
    imgfile = envi.open(hdrfile)
    #read hdr files
    header_hdr = envi.read_envi_header(hdrfile)
    #create cube
    cube = imgfile[:,:,:]
    #replace "nodata" values with np.nan
    #cube_np_nan = np.where(cube < cube.max(), cube, np.nan)
    #create index for wavelengths
    cube_masked = np.ma.masked_where(cube == cube.max(), cube, copy=True)
    names = header_hdr['wavelength']
    return(cube_masked, names)

This function initialize and empty pandas dataframe, then read every slice (image) of the hyperspectral cube, flatten the image, convert to a pandas's series and append to the initial dataframe

In [4]:
def CUBE_np2df(np_cube, names):
    cube_df = pd.DataFrame()
    for i in range(len(names)):
        img_slice = np_cube[:,:,i]
        series = pd.Series(data=img_slice.flatten(), name=names[i])
        cube_df[series.name] = series        
    return(cube_df)

Following functions are dedicated to save the dataframe in csv and hdf format and print the timings. Both files are saved in the same folder of the hdr file

In [5]:
def CUBE_df2csv(cube_df, hdrfile):
    dfname = path_leaf(hdrfile)
    savepath = os.path.dirname(hdrfile)
    savename = savepath + '/' + dfname + '.csv'
    start = timer()
    %time cube_df.to_csv(savename, index=False)
    end = timer()
    print(savename, ' csv exported in: ', savepath, 'in: ', end-start, 's')
    return(end-start)

def CUBE_df2hdf(cube_df, hdrfile):
    dfname = path_leaf(hdrfile)
    savepath = os.path.dirname(hdrfile)
    savename = savepath + '/' + dfname + '.hdf'
    start = timer()
    %time cube_df.to_hdf(savename, 'df')
    end = timer()
    print(savename, ' hdf exported in: ', savepath, 'in: ', end-start, 's')

Main function simply call all the functions.

In [6]:
def main(hdrfile):
    cube_np_nan, names = img2np(hdrfile)
    cube_df = CUBE_np2df(cube_np_nan, names)
    
    print('Saving to csv')
    #export cube to csv
    #csv_time = CUBE_df2csv(cube_df, hdrfile) UNCOMMENT TO SAVE CSV
    print('Saving to hdf')
    #export cube to hdf
    hdf_time = CUBE_df2hdf(cube_df, hdrfile)
    #return(cube_np_nan, cube_df)

Main code initialize a parser to accept the path to the hdr files, and if not given, ask interactively to browse to the file. Then run the main function. Note that this cannot be executed yet on jupiter because of the argparser.

In [ ]:
if __name__ == "__main__":
    parser = ArgumentParser()
    parser.add_argument('--hdrf', help='Select*.hdr file.')
    args = parser.parse_args()
    hdrfile = args.hdrf
    
    if args.hdrf is None:
        root = Tk()
        root.withdraw()
        hdrfile = filedialog.askopenfilename(parent=root,initialdir=os.getcwd(),title="Please select hdr file:",
                                             filetypes= (('hdr files', '*.hdr'), ('all files', '*.*)))')))
        print('hdr file selected: ', hdrfile)        
    else:
        modpath = args.wdir

main(hdrfile)